# Problem Set #7

# Hyun Ki Kim

# 03/05/2018

In [1]:
import numpy as np
import pandas as pd
import sklearn
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from pylab import rcParams
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.tree import export_graphviz
import graphviz
from sklearn import svm

/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
auto = pd.read_csv('Auto.csv', na_values='?')
auto = auto.dropna()
auto['constant'] = 1
auto['mpg_high'] = auto['mpg'].apply(lambda x: 1 if x >= auto['mpg'].median() else 0)

In [3]:
X = auto[["constant","cylinders", "displacement", \
                 "horsepower", "weight", "acceleration", "year", "origin"]].values
y = auto['mpg_high'].values

# 1. Classifier "horse" race

# (a)

In [4]:
k=4
kf = KFold(n_splits=k, shuffle=True, random_state=15)
kf.get_n_splits(X)
MSE_vec_kf = np.zeros(k)
y_test_vec = np.array([])
y_pred_vec = np.array([])

k_ind = int(0)
for train_index, test_index in kf.split(X):
    print('k index:', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    y_test_vec = np.append(y_test_vec, y_test)
    LogReg = LogisticRegression(fit_intercept=True)
    LogReg.fit(X_train, y_train)
    y_pred = LogReg.predict(X_test)
    y_pred_vec = np.append(y_pred_vec, y_pred)
    MSE_vec_kf[k_ind] = ((y_test - y_pred) ** 2).mean()
    print('\n',classification_report(y_test, y_pred, digits=3))
    print('MSE for test set', k_ind, ':', MSE_vec_kf[k_ind])
    print("______________________________________________________")
    print("\n")
    k_ind += 1

MSE_kf = MSE_vec_kf.mean()
MSE_kf_std = MSE_vec_kf.std()
print('Test estimate MSE k-fold:', MSE_kf)
print()
print('Test estimate MSE standard err:', MSE_kf_std)

k index: 0

              precision    recall  f1-score   support

          0      0.942     0.891     0.916        55
          1      0.870     0.930     0.899        43

avg / total      0.910     0.908     0.908        98

MSE for test set 0 : 0.0918367346939
______________________________________________________


k index: 1

              precision    recall  f1-score   support

          0      0.878     0.915     0.896        47
          1      0.918     0.882     0.900        51

avg / total      0.899     0.898     0.898        98

MSE for test set 1 : 0.102040816327
______________________________________________________


k index: 2

              precision    recall  f1-score   support

          0      0.848     0.867     0.857        45
          1      0.885     0.868     0.876        53

avg / total      0.868     0.867     0.867        98

MSE for test set 2 : 0.132653061224
______________________________________________________


k index: 3

              precision 

# (b)

In [5]:
B = 20
m = 2
random_forest = RandomForestClassifier(n_estimators=B, max_features=m, bootstrap=True,
                                  oob_score=True, random_state=25)
random_forest.fit(X, y)

random_forest.score(X, y)
rf_pred = random_forest.oob_decision_function_
MSE_rf = pd.DataFrame({'ypred' : rf_pred.T[1], 'yvals': y})
MSE_rf['ypred'] = MSE_rf['ypred'].apply(lambda x: 1 if x >= 0.5 else 0)
MSE_model = mean_squared_error(MSE_rf['yvals'], MSE_rf['ypred'])
print('The MSE of the random forest model:',MSE_model)

The MSE of the random forest model: 0.0790816326531


In [6]:
MSE_rf_0 = MSE_rf[MSE_rf['ypred'] < 0.5]
MSE_0 = mean_squared_error(MSE_rf_0['yvals'], MSE_rf_0['ypred'])
print('The MSE of the mpg_high = 0:',MSE_0)

The MSE of the mpg_high = 0: 0.0725388601036


In [7]:
MSE_rf_1 = MSE_rf[MSE_rf['ypred'] >= 0.5]
MSE_1 = mean_squared_error(MSE_rf_1['yvals'], MSE_rf_1['ypred'])
print('The MSE of the mpg_high = 1:',MSE_1)

The MSE of the mpg_high = 1: 0.0854271356784


# (c)

In [8]:
k=4
kf = KFold(n_splits=k, shuffle=True, random_state=15)
kf.get_n_splits(X)
SVM_MSE = np.zeros(k)
SVM_error_0 = np.zeros(k)
SVM_error_1 = np.zeros(k)

k_ind = int(0)
for train_index, test_index in kf.split(X):
    print('k index:', k_ind)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    y_test_vec = np.append(y_test_vec, y_test)
    svc = svm.SVC(kernel='rbf', gamma = 0.2, C=1)
    svc.fit(X_train, y_train)
    y_pred = svc.predict(X_test)
    error = y_test != y_pred
    error_all_class = error.mean()
    error_0 = ((y_pred == 0) * error).sum() / (y_pred == 0).sum() 
    error_1 = ((y_pred == 1) * error).sum() / (y_pred == 1).sum()
    SVM_error_0[k_ind] = error_0
    SVM_error_1[k_ind] = error_1
    SVM_MSE[k_ind] = ((y_test - y_pred) ** 2).mean()
    print('\n',classification_report(y_test, y_pred, digits=3))
    print('MSE for test set', k_ind, ':', MSE_vec_kf[k_ind])
    print("______________________________________________________")
    print("\n")
    k_ind += 1

SVM_error_0 = SVM_error_0[~np.isnan(SVM_error_0)]
print('The average error rate of mpg_high = 0:', SVM_error_0.mean())
print("\n")
SVM_error_1 = SVM_error_1[~np.isnan(SVM_error_1)]
print('The average error rate of mpg_high = 1:', SVM_error_1.mean())
print("\n")
print('The average MSE of the SVM model:', SVM_MSE.mean())

k index: 0

              precision    recall  f1-score   support

          0      1.000     0.036     0.070        55
          1      0.448     1.000     0.619        43

avg / total      0.758     0.459     0.311        98

MSE for test set 0 : 0.0918367346939
______________________________________________________


k index: 1

              precision    recall  f1-score   support

          0      0.480     1.000     0.648        47
          1      0.000     0.000     0.000        51

avg / total      0.230     0.480     0.311        98

MSE for test set 1 : 0.102040816327
______________________________________________________


k index: 2

              precision    recall  f1-score   support

          0      0.469     1.000     0.638        45
          1      1.000     0.038     0.073        53

avg / total      0.756     0.480     0.332        98

MSE for test set 2 : 0.132653061224
______________________________________________________


k index: 3

              precision 

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in long_scalars
/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


# (d)

Random forest model is the best predictor of mpg_high becuase it has the lowest MSE.